In [2]:
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, LabelEncoder


In [3]:
dataset_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))), 'datasets')

##### Datos a usar

- V2 --> dataset limpio
- V4 --> variables construidas a partir de otras + agrupacion de violencias absorbidas
- V5 --> eliminación de las variables cn poca información


##### Pipeline

1. cargar dataset
2. reemplazar NS/NC en convive dataset por NA
3. separar NA de convive, serán el dataset "no visto"
4. separar train-test
5. hace modificaciones para svm en train, test, y dataset final: Apply scikit's OneHotEncoder with the handle_unknown parameter set to "ignore"

##### Modificaiones para SVM

1. pasar datetime a timestamp y escalar --> OK
2. escalar la variable edad --> OK
3. pasar edad a categórica (en una columna distinta) --> OK
4. reemplazar NS/NC en convive por NA
5. pasar todas las categóricas a dummy:
   1. edad, momento del día y estación del año con un encoder ordinal
   2. el resto con one hot encoder excepto convive que lo paso cuando armo el modelo
6. borrar las categóricas para hacer más pequeño el dataset




In [29]:
# 1 cargar datasets

llamados_v2= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v2.xlsx'), parse_dates=['llamado_fecha_hora'])
llamados_v4= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v4.xlsx'), parse_dates=['llamado_fecha_hora'])
llamados_v5= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v5.xlsx'), parse_dates=['llamado_fecha_hora'])


In [30]:
todos_datasets = [llamados_v2, llamados_v4, llamados_v5]
datasets_45 = [llamados_v4, llamados_v5]

In [31]:
scaler = MinMaxScaler()
encoder = OrdinalEncoder()
label_encoder = LabelEncoder()

In [32]:
# 2 reemplazar todos los NS/NC en convive por na
for dataset in todos_datasets:

    # Replace the specified values with NaN
    dataset.loc[:, 'victima_convive_agresor'] = dataset['victima_convive_agresor'].replace({'NS/NC': pd.NA})

In [36]:
# 3 separar los na de convive en nuevo dataset

# Create a copy of the dataset where Column F has a value NaN
llamados_v2_no_visto = llamados_v2[llamados_v2['victima_convive_agresor'].isna()].copy(deep=True)
llamados_v2_train_test = llamados_v2[~llamados_v2['victima_convive_agresor'].isna()].copy(deep=True)

llamados_v4_no_visto = llamados_v4[llamados_v4['victima_convive_agresor'].isna()].copy(deep=True)
llamados_v4_train_test = llamados_v4[~llamados_v4['victima_convive_agresor'].isna()].copy(deep=True)

llamados_v5_no_visto = llamados_v5[llamados_v5['victima_convive_agresor'].isna()].copy(deep=True)
llamados_v5_train_test = llamados_v5[~llamados_v5['victima_convive_agresor'].isna()].copy(deep=True)



In [43]:
llamados_v2_no_visto.to_excel(os.path.join(dataset_dir, 'xlsx/llamados_v2_no_visto.xlsx'), index=False)
llamados_v4_no_visto.to_excel(os.path.join(dataset_dir, 'xlsx/llamados_v4_no_visto.xlsx'), index=False)
llamados_v5_no_visto.to_excel(os.path.join(dataset_dir, 'xlsx/llamados_v5_no_visto.xlsx'), index=False)


In [41]:
del llamados_v2, llamados_v4, llamados_v5
del llamados_v4_no_visto, llamados_v5_no_visto, llamados_v2_no_visto

In [ ]:
# 4 separar train_test

In [43]:
# 1. fecha hora a timestamp y escalado

for dataset in todos_datasets:
    dataset['timestamp_encoded'] = encoder.fit_transform(dataset[['llamado_fecha_hora']])
    dataset['timestamp_scaled'] = scaler.fit_transform(dataset[['timestamp_encoded']])
    dataset.drop('timestamp_encoded', axis=1, inplace=True)
    dataset.drop('llamado_fecha_hora', axis=1, inplace=True )

In [44]:
# 2. escalar edades
for dataset in todos_datasets:
    dataset['victima_edad_escalada'] = scaler.fit_transform(dataset[['victima_edad']])
    dataset['llamante_edad_escalada'] = scaler.fit_transform(dataset[['llamante_edad']])

In [45]:
# 3. pasar edad a categórica


def categoria_edad (x):
    if (x >= 0) and (x <= 11) :
        return 'Niñez'
    elif (x >= 12) and (x <=18):
        return 'Adolescencia'
    elif (x >= 19) and (x <=30):
        return 'Juventud'
    elif (x>=31) and (x<=65) :
        return 'Vejez'
    elif x>=66:
        return 'Vejez mayor'
    else:
        return 'NS/NC'

for dataset in todos_datasets:
    dataset['victima_edad_cat'] = \
        dataset.victima_edad.apply(categoria_edad)
    dataset['llamante_edad_cat'] = \
        dataset.llamante_edad.apply(categoria_edad)
    dataset.drop(['victima_edad','llamante_edad'], axis=1, inplace=True)




In [78]:
# 4. reemplazar NS/NC de convive por NA


for dataset in todos_datasets:

    # Replace the specified values with NaN
    dataset.loc[:, 'victima_convive_agresor'] = dataset['victima_convive_agresor'].replace({' NS/NC': pd.NA})

In [53]:
# 5.1 pasar edad_cat, momento_dia (V4, V5) y estacion_del_año (V4, V5) a dummy con un encoder ordinal


for dataset in todos_datasets:
    dataset['victima_edad_cat_dummy'] = label_encoder.fit_transform(dataset['victima_edad_cat'])
    dataset['llamante_edad_cat_dummy'] = label_encoder.fit_transform(dataset['llamante_edad_cat'])
    dataset.drop(['victima_edad_cat', 'llamante_edad_cat'], axis=1, inplace=True)

for dataset in datasets_45:
    dataset['momento_dummy'] = label_encoder.fit_transform(dataset['momento_dia'])
    dataset['estacion_dummy'] = label_encoder.fit_transform(dataset['estacion_del_año'])
    dataset.drop(['estacion_del_año','momento_dia',], axis=1, inplace=True)


In [ ]:
# escalar las variables ordinales edad, momento del día y estación

for dataset in todos_datasets:
    dataset['victima_edad_dummy_scaled'] = scaler.fit_transform(dataset[['victima_edad_cat_dummy']])
    dataset['llamante_edad_dummy_scaled'] = scaler.fit_transform(dataset[['llamante_edad_cat_dummy']])
    dataset.drop('victima_edad_cat_dummy', axis=1, inplace=True)
    dataset.drop('llamante_edad_cat_dummy', axis=1, inplace=True )

for dataset in datasets_45:
    dataset['momento_dummy_scaled'] = scaler.fit_transform(dataset[['momento_dummy']])
    dataset['estacion_dummy_scaled'] = scaler.fit_transform(dataset[['estacion_dummy']])
    dataset.drop('momento_dummy', axis=1, inplace=True)
    dataset.drop('estacion_dummy', axis=1, inplace=True )


In [ ]:
# sacar todos los casos vacíos de convive, esos serán el dataset aparte final a predecir con el mejor modelo
# encodear el resto del dataset, todo menos y_convive

def one_hot_encoder(dataset):
    columna_excluir = 'victima_convive_agresor'
    columnas_cat = dataset.select_dtypes(include=['object']).columns.tolist()
    columnas_cat.remove(columna_excluir)

    # Apply one-hot encoding to object columns excluding the chosen one
    df_encoded = pd.get_dummies(df, columns=object_columns)

    # Display the DataFrame with one-hot encoded columns
    print(df_encoded)


### Experimentos usando los datasets preparados para svm:

- las variaciones son en las columnas de edad, en los datasets que tengo, y en los kernels de SVM

##### Prueba 1, dataset V2, V3, V4
- edad categórica pasada a dummy se va
- edad numérica se queda pero solo los datos completos de ambas
- corro SVN c =/= kernels

##### Prueba 2, dataset V2, V3, V4
- edad categórica pasada a dummy se va
- edad numérica se queda pero solo los datos completos de victima
- corro SVN c =/= kernels

##### Prueba 3, dataset V2, V3, V4
- edad categórica pasada a dummy se va
- edad numérica se queda pero solo los datos completos de llamante
- corro SVN c =/= kernels

##### Prueba 4, dataset V2, V3, V4
- edad numérica se va
- edad categórica pasada a dummy se queda
- corro SVN c =/= kernels